In [69]:
# Imports
import pandas as pd
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from time import sleep

# Install the webdriver_manager pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install asyncio

In [38]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/matt/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


In [85]:
def start_kayak(city_from, city_to, date_start, date_end):
    """City codes - it's the IATA codes!
    Date format -  YYYY-MM-DD"""
    
    # Starting kayak chrome 
    kayak = ('https://www.kayak.co.uk/flights/' + city_from + '-' + city_to +
             '/' + date_start + 'sort=bestflight_a?sort=price_a&fs=stops=~0')
    driver.get(kayak)
    sleep(5) # sleep to allow the website to load
    
    # If the privacy modal opens, click accept
    try:
        accept_button_xpath = '//button[contains(@id, "-accept")]'
        accept_button = driver.find_element(By.XPATH, accept_button_xpath)
        accept_button.click()
        sleep(1)
    except Exception as e:
        # try clicking accept all on the other flavour of privacy modal
        # try:
        print(e)
        
    actions = ActionChains(driver)
    
    # Click on filter to include flights only
    try: 
        plane_checkbox_xpath = '//div[contains(@id, "-transportation_plane-check-icon")]'
        plane_only_xpath = '//button[contains(@id, "-transportation_plane-only")]'
        plane_checkbox_div = driver.find_element(By.XPATH, plane_checkbox_xpath)
        plane_only_button = driver.find_element(By.XPATH, plane_only_xpath)
        
        actions.move_to_element(plane_checkbox_div)
        actions.pause(5)
        actions.click(plane_only_button)
        actions.perform()
        sleep(1)
    except Exception as e:
        print(e)
    
    

In [65]:
# functions to obtain each property of a dictionary which will then be the exported csv file

"""
{
origin: char(3)
destination: char(3)
time_start: int
time_end: int
date_start: date
date_end: date
price: int
href: string;
}
"""

# need to find a way to scrape a single flight, then iterate

def scrape_hrefs():
    """
    scrape the hrefs on the page
    """
    href_xpath = '//a[contains(@aria-label, "View Deal:")]//span[@class="price-text"]//ancestor::a[contains(@aria-label, "View Deal:")]'
    anchors = driver.find_elements(By.XPATH, href_xpath)
    
    def get_href(anchor):
        return anchor.get_attribute("href")
    
    print(len(anchors))
    
    return list(map(get_href, anchors)) 

def scrape_prices(): 
    """
    scrape the prices on the page
    """
    
    price_xpath = '//a[contains(@aria-label, "View Deal:")]//span[@class="price-text"]'
    spans = driver.find_elements(By.XPATH, price_xpath)
    
    def get_price(span):
        return span.text;
    
    print(len(spans))
    
    return list(map(get_price, spans))

def scrape_start_times():
    """
    scrape the depart times
    """
    
    start_times_xpath = '//a[contains(@aria-label, "View Deal:")]//preceding::div[@class="section stacked-carriers"]//following-sibling::div[@class="section times"]//descendant::span[@class="depart-time base-time"]'
    spans = driver.find_elements(By.XPATH, start_times_xpath)
    
    def get_time(span):
        return span.text
    
    print(len(spans))
    
    return list(map(get_time, spans))

def scrape_end_times():
    """
    scrape the depart times
    """
    
    start_times_xpath = '//a[contains(@aria-label, "View Deal:")]//preceding::div[@class="section stacked-carriers"]//following-sibling::div[@class="section times"]//descendant::span[@class="arrival-time base-time"]'
    spans = driver.find_elements(By.XPATH, start_times_xpath)
    
    def get_time(span):
        return span.text
    
    print(len(spans))
    
    return list(map(get_time, spans))

def scrape_durations():
    """
    scrape durations because of time zone differences
    """
    durations_xpath = '//a[contains(@aria-label, "View Deal:")]//preceding::div[@class="section stacked-carriers"]//following-sibling::div[@class="section duration allow-multi-modal-icons"]//child::div[@class="top"]'
    divs = driver.find_elements(By.XPATH, durations_xpath)
    
    def get_duration(div):
        return div.text
    
    print(len(divs))
    
    return list(map(get_duration, divs))


In [89]:
start_kayak("LHR", "CDG", "2022-03-15", "2022-03-22")

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[contains(@id, "-accept")]"}
  (Session info: chrome=98.0.4758.102)



In [81]:
depart_times = scrape_start_times()
arrival_times = scrape_end_times()
prices = scrape_prices()
durations = scrape_durations()
links = scrape_hrefs()

12
12
12
12
12


In [82]:
# Create dataframe
df = pd.DataFrame(np.array([depart_times, arrival_times, prices, durations, links]).transpose(),
                   columns=['departure_time', 'arrival_time', 'price', 'duration', 'link'])

In [83]:
df

,departure_time,arrival_time,price,duration,link
0,23:30,08:35,£23,8h 05m,https://www.kayak.co.uk/book/flight?code=KaECX...
1,18:01,21:23,£33,2h 22m,https://www.kayak.co.uk/book/flight?code=KaECX...
2,20:30,22:50,£43,1h 20m,https://www.kayak.co.uk/book/flight?code=KaECX...
3,07:15,09:30,£46,1h 15m,https://www.kayak.co.uk/book/flight?code=KaECX...
4,17:20,19:45,£46,1h 25m,https://www.kayak.co.uk/book/flight?code=KaECX...
5,14:50,17:10,£49,1h 20m,https://www.kayak.co.uk/book/flight?code=KaECX...
6,11:30,13:50,£63,1h 20m,https://www.kayak.co.uk/book/flight?code=KaECX...
7,13:05,15:25,£72,1h 20m,https://www.kayak.co.uk/book/flight?code=KaECX...
8,17:35,19:55,£83,1h 20m,https://www.kayak.co.uk/book/flight?code=KaECX...
9,08:35,11:00,£105,1h 25m,https://www.kayak.co.uk/book/flight?code=KaECX...
